In [14]:
import os
import nltk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer




In [15]:
comentarios_Peliculas = pd.read_csv(os.path.join("data/imdb_dataset.csv"),encoding="latin-1").sample(10000,replace=False) 

In [16]:
print(comentarios_Peliculas)

                                                  Review Label
969    Director/lead Larry Bishop tried way, way too ...   neg
44998  The year is 1896.Jeff Webster (James Stewart) ...   pos
21800  There are few films that deal with things that...   pos
11372  First, let me state that I have no idea who No...   neg
26214  If this movie was about a fictional character,...   neg
...                                                  ...   ...
42124  We toss around the term "superstar" way too li...   pos
39088  till HBO began rerunning it this month. I reme...   pos
22663  let's face it, you know what to expect when yo...   pos
20790  When I first saw Colleen Moore it was in the e...   pos
10939  The other day I showed my boyfriend a great mo...   neg

[10000 rows x 2 columns]


In [32]:
"""LIMPIAR DATOS"""
cleanWords=[]
for sent in comentarios_Peliculas['Review']:
    stop = set(stopwords.words('english') + list(string.punctuation))
    cleanWords.append(" ".join([str(i) for i in word_tokenize(sent.lower()) if i not in stop]))
print(cleanWords[:5])

        
        



["director/lead larry bishop tried way way hard hell ride movie wants edgy witty provocative outlandish biting seemingly quentin tarantino/rob zombie style 's edgy references seem forced dialog tries clever fails humor never funny nice try setting gritty tone 'd care characters story remotely succeed. br br 're left cool harleys pretty girls surrounding bunch tired old shape `` bad boys '' looks like attempt modernized sergio leone western movie make newer generations interested 60s 70s films kudos rather boring irrelevant believe place style substance movie", "year 1896.jeff webster james stewart n't like people.there 's one friend 's got 's ben tatum walter brennan old sympathetic man.they 're driving cattle herd them.that would key richness.in skagway run trouble sheriff gannon john mcintire takes cattle.now jeff get back drive u.s. canadian border dawson.now group people like ladies ronda castle ruth roman renee vallon corinne calvet .there two men get gold business.anthony mann 's

In [33]:
"""ONe hot coding"""

cv = CountVectorizer()
cv_matrix = cv.fit_transform(cleanWords)
oneHotCoding = np.sign(cv_matrix.toarray())
print(oneHotCoding)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [34]:
"""Vocabulario del problema"""

vocabulary = cv.get_feature_names()
with open('vocabularioProblema.txt','w')  as f:
    for item in vocabulary:
        f.write("%s\n" % item)



In [35]:
new_data = pd.DataFrame(oneHotCoding,columns=vocabulary)
print(new_data.shape)
new_data.head()

(10000, 52302)


,00,000,00000001,001,003830,007,0079,0083,009,00am,...,zy,zzzz,zzzzzzzzzzzzzzzzzz,zã,â½,â¾,ã¼ber,ã¼bermensch,ã½,ã½s
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
"""entrenamiento"""
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score,f1_score

In [45]:
x = new_data[vocabulary]
y = comentarios_Peliculas['Label'].apply(lambda catalogedComment:1 if catalogedComment=="pos" else 0)
print(x.shape,y.shape)
y[:6]

(10000, 52302) (10000,)


969      0
44998    1
21800    1
11372    0
26214    0
3528     0
Name: Label, dtype: int64

In [46]:
def set_training_test_groups(x,y):
    size_train = int((x.shape[0]*0.7))
    x_train = x.loc[:size_train-1]
    y_train = y[:size_train]
    x_test = x.loc[size_train:]
    y_test = y[size_train:]
    return x_train,x_test,y_train,y_test

In [47]:
x_train,x_test,y_train,y_test = set_training_test_groups(x,y)
print("training x->",x_train.shape,"y->",y_train.shape)
print("test x->",x_test.shape,"y->",y_test.shape)

training x-> (7000, 52302) y-> (7000,)
test x-> (3000, 52302) y-> (3000,)


In [48]:
"metodo de entrenamiento con arboles de decision"
"entrenador Ensemble Learning"

from sklearn.ensemble import RandomForestClassifier

tree_v1 = RandomForestClassifier(n_estimators=40)

tree_v1

RandomForestClassifier(n_estimators=40)

In [49]:
"entrenar"
tree_v1.fit(x_train,y_train)


RandomForestClassifier(n_estimators=40)

In [50]:
predictions = tree_v1.predict(x_test)
# métricas de desempeño
# accuracy
print('accuracy del clasificador - version 2 : {0:.2f}'.format(accuracy_score(y_test, predictions)))
# confusion matrix
print('matriz de confusión del clasificador - version 1: \n {0}'.format(confusion_matrix(y_test, predictions)))
# precision 
print('precision del clasificador - version 2 : {0:.2f}'.format(precision_score(y_test, predictions)))
# precision 
print('recall del clasificador - version 2 : {0:.2f}'.format(recall_score(y_test,predictions)))
# f1
print('f1 del clasificador - version 3 : {0:.2f}'.format(f1_score(y_test, predictions)))

accuracy del clasificador - version 2 : 0.82
matriz de confusión del clasificador - version 1: 
 [[1227  241]
 [ 311 1221]]
precision del clasificador - version 2 : 0.84
recall del clasificador - version 2 : 0.80
f1 del clasificador - version 3 : 0.82


In [ ]:
"Guardar el modelo"

